In [ ]:
 import tensorflow as tf
tf.compat.v1.disable_v2_behavior()

import numpy as np

tf.get_logger().setLevel('ERROR')

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
pip install tensorflow-privacy

     |████████████████████████████████| 251 kB 7.2 MB/s 
     |████████████████████████████████| 4.0 MB 75.9 MB/s 
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [ ]:
import tensorflow_privacy
import keras
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

x_train = np.array(x_train, dtype=np.float32) / 255
x_test = np.array(x_test, dtype=np.float32) / 255

x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

y_train = np.array(y_train, dtype=np.int32)
y_test = np.array(y_test, dtype=np.int32)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

assert x_train.min() == 0.
assert x_train.max() == 1.
assert x_test.min() == 0.
assert x_test.max() == 1.


assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_test.shape == (10000, 100)

169017344/169001437 [==============================] - 4s 0us/step


In [ ]:
epochs = 100
batch_size = 250

In [ ]:
l2_norm_clip = 1.0
noise_multiplier = 0.1
num_microbatches = 250
learning_rate = 0.15

if batch_size % num_microbatches != 0:
  raise ValueError('Batch size should be an integer multiple of the number of microbatches')

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 8,
                           strides=2,
                           padding='same',
                           activation='relu',
                           input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Conv2D(32, 4,
                           strides=2,
                           padding='valid',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(2, 1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(100)
])

In [ ]:
optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - ETA: 0s - loss: 4.6016 - acc: 0.0101

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 [==============================] - 68s 1ms/sample - loss: 4.6016 - acc: 0.0101 - val_loss: 4.5941 - val_acc: 0.0137
Epoch 2/100
50000/50000 [==============================] - 47s 939us/sample - loss: 4.5803 - acc: 0.0168 - val_loss: 4.5617 - val_acc: 0.0169
Epoch 3/100
50000/50000 [==============================] - 47s 940us/sample - loss: 4.5363 - acc: 0.0245 - val_loss: 4.5073 - val_acc: 0.0266
Epoch 4/100
50000/50000 [==============================] - 47s 939us/sample - loss: 4.4804 - acc: 0.0304 - val_loss: 4.4517 - val_acc: 0.0320
Epoch 5/100
50000/50000 [==============================] - 47s 938us/sample - loss: 4.4270 - acc: 0.0353 - val_loss: 4.4049 - val_acc: 0.0358
Epoch 6/100
50000/50000 [==============================] - 47s 938us/sample - loss: 4.3804 - acc: 0.0389 - val_loss: 4.3621 - val_acc: 0.0411
Epoch 7/100
50000/50000 [==============================] - 47s 939us/sample - loss: 4.3409 - acc: 0.0439 - val_loss: 4.3221 - val_acc: 0.0473
Epoch 8/100
50000/50

In [ ]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=x_train.shape[0],
                                              batch_size=batch_size,
                                              noise_multiplier=noise_multiplier,
                                              epochs=epochs,
                                              delta= 4e-05)

DP-SGD with sampling rate = 0.5% and noise_multiplier = 0.1 iterated over 20000 steps satisfies differential privacy with eps = 7.2e+05 and delta = 4e-05.
The optimal RDP order is 1.25.
The privacy estimate is likely to be improved by expanding the set of orders.


(720216.0577628489, 1.25)

 DPKerasAdamOptimizer

In [ ]:
optimizer2 = tensorflow_privacy.DPKerasAdamOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [ ]:
model.compile(optimizer=optimizer2, loss=loss, metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=50,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - ETA: 0s - loss: 16.2405 - acc: 0.0108

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 [==============================] - 62s 1ms/sample - loss: 16.2405 - acc: 0.0108 - val_loss: 4.6258 - val_acc: 0.0100
Epoch 2/50
50000/50000 [==============================] - 47s 940us/sample - loss: 5.1622 - acc: 0.0098 - val_loss: 5.4248 - val_acc: 0.0100
Epoch 3/50
50000/50000 [==============================] - 47s 938us/sample - loss: 8.4272 - acc: 0.0105 - val_loss: 8.2673 - val_acc: 0.0100
Epoch 4/50
50000/50000 [==============================] - 47s 940us/sample - loss: 8.3369 - acc: 0.0098 - val_loss: 13.5662 - val_acc: 0.0100
Epoch 5/50
50000/50000 [==============================] - 47s 938us/sample - loss: 8.2061 - acc: 0.0098 - val_loss: 7.2915 - val_acc: 0.0100
Epoch 6/50
50000/50000 [==============================] - 47s 939us/sample - loss: 15.0850 - acc: 0.0096 - val_loss: 16.6405 - val_acc: 0.0100
Epoch 7/50
50000/50000 [==============================] - 47s 939us/sample - loss: 22.5254 - acc: 0.0092 - val_loss: 9.1942 - val_acc: 0.0100
Epoch 8/50
50000/5000

DPKerasAdagradOptimizer

In [ ]:
optimizer3 = tensorflow_privacy.DPKerasAdagradOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate)

loss = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, reduction=tf.losses.Reduction.NONE)

In [ ]:
model.compile(optimizer=optimizer3, loss=loss, metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=50,
          validation_data=(x_test, y_test),
          batch_size=batch_size)

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - ETA: 0s - loss: 113.4696 - acc: 0.0098

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


50000/50000 [==============================] - 63s 1ms/sample - loss: 113.4696 - acc: 0.0098 - val_loss: 174.5423 - val_acc: 0.0100
Epoch 2/50
50000/50000 [==============================] - 47s 941us/sample - loss: 113.0945 - acc: 0.0095 - val_loss: 173.9655 - val_acc: 0.0100
Epoch 3/50
50000/50000 [==============================] - 47s 941us/sample - loss: 112.3110 - acc: 0.0096 - val_loss: 174.6044 - val_acc: 0.0100
Epoch 4/50
50000/50000 [==============================] - 47s 940us/sample - loss: 111.9937 - acc: 0.0095 - val_loss: 174.4421 - val_acc: 0.0100
Epoch 5/50
50000/50000 [==============================] - 47s 940us/sample - loss: 110.9883 - acc: 0.0094 - val_loss: 174.6652 - val_acc: 0.0100
Epoch 6/50
50000/50000 [==============================] - 47s 940us/sample - loss: 110.5566 - acc: 0.0089 - val_loss: 174.0176 - val_acc: 0.0099
Epoch 7/50
50000/50000 [==============================] - 47s 941us/sample - loss: 110.1707 - acc: 0.0089 - val_loss: 174.9289 - val_acc: 0.010